# Calibrating WFI Data with RomanCal 

***

## Imports
 Libraries used
- *romancal* for running the processing pipeline
- *roman_datamodels* for opening Roman WFI ASDF files
- *s3fs* for streaming files from an S3 bucket

In [ ]:
import roman_datamodels as rdm
from romancal.pipeline import ExposurePipeline
import s3fs

## Introduction
The purpose of this notebook is to calibrate Level 1 (L1) data with the Roman WFI science calibration pipeline RomanCal (Python package name `romancal`) to produce Level 2 (L2) data. 

Details about the roman data levels can be found [here](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). Broadly speaking Level 1 is three-dimensional data cube (one dimension for time and two dimensions for image coordinates) that represents a single uncalibrated ramp exposure in units of Data Numbers (DN).  More specifically, it is shaped as an array with shape (N resultants, 4096 image rows, 4096 image columns). The Level 2 WFI data are calibrated rate images in instrumental units of DN / second. 

***

## Tutorial Data
In this tutorial, we use Level 1 WFI data files simulated with `romanisim`. The files are stored in the science platform S3 bucket. For more information on how to access these data, see the Data Discovery and Access tutorial.

## Run romancal on L1 Data
To run `romancal` on the L1 data, there are two options:
1. You can use the exposure-level pipeline to run all steps (basic), or
2. You can run one or more individual steps (advanced).

### Basic Example: Full Pipeline

The input file for our example is a WFI L1 ASDF file. We must copy the file to our local storage as the RomanCal exposure pipeline currently only works with an input file on disk. Individual pipeline steps (described further below) can run directly on datamodels in memory.

In [ ]:
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
fs = s3fs.S3FileSystem()

asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.5_DP0.5_PA0/r0000101001001001001_01101_0001_WFI01_uncal.asdf'
_ = fs.get(asdf_file_uri, '.')

Let's start with basic example of running the "complete" pipeline. We do skip the final two steps in the pipeline in this example (source detection and tweakreg) for two reasons: first, the source detection step on our example takes a long time to run due to the density of sources; and second, source detection and tweakreg are meant to align the image to the Gaia astrometric frame, but the sources in our simulated image are not real and do not correspond to Gaia sources. There are other optional parameters that may be set for individual steps in a similar manner, and more information can be found in the [romancal documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html).

The save_results optional parameter will save the resulting file to your local disk. You can enable this by setting the value to True. In our example, we will keep the output calibrated result in memory instead.

In [ ]:
pipe = ExposurePipeline(save_results=False, steps={'source_detection': {'skip': True}, 'tweakreg': {'skip': True}})
result = pipe.process('r0000101001001001001_01101_0001_WFI01_uncal.asdf')

The output from the exposure pipeline is currently returned as a list. You will need to index the results variable with a `[0]` to access the datamodel object.

In [ ]:
type(result)

In [ ]:
type(result[0])

In the example above, you can see how we passed optional parameters to individual steps in the pipeline. The pipeline steps in order are:

- `romancal.dq_init.dq_init_step`: Bad pixel masking and data quality initialization
- `romancal.saturation.SaturationStep`: Saturation flagging up-the-ramp
- `romancal.refpix.RefPixStep`: 1/f noise correction
- `romancal.linearity.LinearityStep`: Classic non-linearity correction
- `romancal.dark_current.DarkCurrentStep`: Dark current subtraction
- `romancal.ramp_fitting.ramp_fit_step`: Jump detection and fitting up-the-ramp
- `romancal.assign_wcs.AssignWcsStep`: Initialize the WCS with the pointing information
- `romancal.flatfield.FlatFieldStep`: Apply the flat field to the data
- `romancal.photom.PhotomStep`: Populate photometric calibration information
- `romancal.source_detection.SourceDetectionStep`: Run source detection on the image
- `romancal.tweakreg.TweakRegStep`: Match sources to Gaia and update WCS information

The ramp fitting step fundamentally transforms the datamodel in memory, therefore steps after ramp fitting cannot be applied to a datamodel that has not been run through the ramp fitting step (and likewise steps prior to ramp fitting should not be run on datamodels after the ramp fitting step).

### Advanced Example: Running Individual Pipeline Steps

Now for a mode advanced use case, let's look at an example of updating the WCS based on the pointing information. In this example, let's imagine that we have simulated a field in a L2 calibrated file, and we want to update the distortion model in the file's gwcs object. We can do so by running the individual AssignWcsStep, which we do below using a L2 ASDF file streamed from the S3 bucket. Let's start by opening the L2 file in memory from the S3 bucket:

In [ ]:
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
fs = s3fs.S3FileSystem()

asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.5_DP0.5_PA0/r0000101001001001001_01101_0001_WFI01_cal.asdf'
with fs.open(asdf_file_uri, 'rb') as f:
    dm = rdm.open(f)

Next, let's run AssignWcsStep on the datamodel. Doing so will return an updated datamodel in memory:

In [ ]:
dm.info()

In [ ]:
wcs_step = romancal.assign_wcs.AssignWcsStep()
result = wcs_step.process(dm)

In this case, the output from the step is the datamodel rather than a list:

In [ ]:
type(result)

If we have our own version of the distortion reference file that we want to use rather than the one from CRDS, then we can use that file with the override_distortion optional parameter:

In [ ]:
# wcs_step = romancal.assign_wcs.AssignWcsStep(override_distortion='my_distortion_file.asdf')
# result = wcs_step.process(dm)

Similar override parameters exist for all reference file types. More information on WFI reference file types may be found in the RDox article [CRDS for Reference Files](https://roman-docs.stsci.edu/data-handbook-home/accessing-wfi-data/crds-for-reference-files).

Just as before, we directed the updated datamodel to the variable "result" in active memory. Just like with other datamodel objects we can access the information in it (such as the gwcs object or data array) as we would any other datamodel. If we want to write it to disk, we can do so with the save method. For more information, see the Working with ASDF tutorial. We can also pass this datamodel along to the next step if we want and chain steps together.

## Aditional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this notebook
**Author:** Sanjib Sharma, Roman Telescope Branch.  
**Updated On:** 2024-06-18

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 